In [109]:
import pandas as pd
import numpy as np

#countvectorizer model
import nltk
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

#imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [110]:
df = pd.read_csv("../data/goodreads_final_bagowords.csv")
df.head()

,url,bookTitle,bookImage,bookDesc,bookRating,ratingCount,reviewCount,Genre,pageCount,Author,lang,title_key_words,clean_keywords,sentiment_polarity,sentiment_subjectivity,genre_bag,bag_of_words
0,https://www.goodreads.com/book/show/25743.The_...,The Slave,https://i.gr-assets.com/images/S/compressed.ph...,Four years after the Chmielnicki massacres of ...,4.19,2886,259,Fiction,320,Isaac Bashevis Singer,en,['slave'],"['four', 'years', 'chmielnicki', 'massacres', ...",0.042338,0.341429,['fiction'],fiction slave four years chmielnicki massacres...
1,https://www.goodreads.com/book/show/13547289-t...,Tales from a Not-So-Smart Miss Know-It-All,https://i.gr-assets.com/images/S/compressed.ph...,Nikki Maxwell authors an advice column for the...,4.39,25705,806,Education,368,Rachel Renée Russell,en,"['tales', 'smart', 'miss', 'know']","['nikki', 'maxwell', 'authors', 'advice', 'col...",-0.008807,0.450042,['education'],education tales smart miss know nikki maxwell ...
2,https://www.goodreads.com/book/show/21899546-l...,"Love, in English",https://i.gr-assets.com/images/S/compressed.ph...,He’s thirty-eight. I’m twenty-three.He speaks ...,4.07,15358,2024,Romance,439,Karina Halle,en,"['love', 'english']","['thirty', 'eight', 'twenty', 'three', 'speaks...",0.175129,0.475090,['romance'],romance love english thirty eight twenty three...
3,https://www.goodreads.com/book/show/31328891-s...,Stuck in the Game,https://i.gr-assets.com/images/S/compressed.ph...,After a terrible car accident puts seventeen-y...,3.84,277,59,Fantasy,226,Christopher Keene,en,"['stuck', 'game']","['terrible', 'car', 'accident', 'puts', 'seven...",-0.151667,0.438819,['fantasy'],fantasy stuck game terrible car accident puts ...
4,https://www.goodreads.com/book/show/10507293-t...,The Selection,https://i.gr-assets.com/images/S/compressed.ph...,"For thirty-five girls, the Selection is the ch...",4.12,1004868,53737,Young Adult,336,Kiera Cass,en,['selection'],"['thirty', 'five', 'girls', 'selection', 'chan...",0.126263,0.573148,['youngadult'],youngadult selection thirty five girls selecti...


## EDA

In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   url                     5000 non-null   object 
 1   bookTitle               5000 non-null   object 
 2   bookImage               5000 non-null   object 
 3   bookDesc                5000 non-null   object 
 4   bookRating              5000 non-null   float64
 5   ratingCount             5000 non-null   int64  
 6   reviewCount             5000 non-null   int64  
 7   Genre                   5000 non-null   object 
 8   pageCount               5000 non-null   int64  
 9   Author                  5000 non-null   object 
 10  lang                    5000 non-null   object 
 11  title_key_words         5000 non-null   object 
 12  clean_keywords          5000 non-null   object 
 13  sentiment_polarity      5000 non-null   float64
 14  sentiment_subjectivity  5000 non-null   

In [112]:
df["bookTitle"].nunique()

5000

In [113]:
df["Author"].nunique()

2667

In [114]:
df["Genre"].nunique()

46

## Count Vectorizer

In [115]:
#CountVectorizer model
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

cosine_sim = cosine_similarity(count_matrix, count_matrix)
print(cosine_sim)

indices = pd.Series(df['bookTitle'])

[[1.         0.         0.06437828 ... 0.04651303 0.01560216 0.02575131]
 [0.         1.         0.02417547 ... 0.01455556 0.0146474  0.07252642]
 [0.06437828 0.02417547 1.         ... 0.10380685 0.04178472 0.06896552]
 ...
 [0.04651303 0.01455556 0.10380685 ... 1.         0.02515773 0.06228411]
 [0.01560216 0.0146474  0.04178472 ... 0.02515773 1.         0.08356943]
 [0.02575131 0.07252642 0.06896552 ... 0.06228411 0.08356943 1.        ]]


In [116]:
test = count_matrix.toarray()
test


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [117]:
len(test[0])

35863

In [118]:
sum(test[0])

50

In [119]:
def recommend(bookTitle, cosine_sim = cosine_sim):
    recommended_book = []
    idx = indices[indices == bookTitle].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[1:11].index)
    
    for i in top_10_indices:
        recommended_book.append(list(df['bookTitle'])[i])
        
    return recommended_book

In [120]:
recommend('To Kill a Mockingbird')

['Go Set a Watchman',
 'Lady of Hay',
 'Our Town',
 'The Grapes of Wrath',
 'Manchild in the Promised Land',
 'Jimmy Corrigan: The Smartest Kid on Earth',
 'The Gift of the Magi',
 'Jennie Gerhardt',
 'Stellaluna',
 'Lila']

## TF- IDF

In [121]:
df.head()

,url,bookTitle,bookImage,bookDesc,bookRating,ratingCount,reviewCount,Genre,pageCount,Author,lang,title_key_words,clean_keywords,sentiment_polarity,sentiment_subjectivity,genre_bag,bag_of_words
0,https://www.goodreads.com/book/show/25743.The_...,The Slave,https://i.gr-assets.com/images/S/compressed.ph...,Four years after the Chmielnicki massacres of ...,4.19,2886,259,Fiction,320,Isaac Bashevis Singer,en,['slave'],"['four', 'years', 'chmielnicki', 'massacres', ...",0.042338,0.341429,['fiction'],fiction slave four years chmielnicki massacres...
1,https://www.goodreads.com/book/show/13547289-t...,Tales from a Not-So-Smart Miss Know-It-All,https://i.gr-assets.com/images/S/compressed.ph...,Nikki Maxwell authors an advice column for the...,4.39,25705,806,Education,368,Rachel Renée Russell,en,"['tales', 'smart', 'miss', 'know']","['nikki', 'maxwell', 'authors', 'advice', 'col...",-0.008807,0.450042,['education'],education tales smart miss know nikki maxwell ...
2,https://www.goodreads.com/book/show/21899546-l...,"Love, in English",https://i.gr-assets.com/images/S/compressed.ph...,He’s thirty-eight. I’m twenty-three.He speaks ...,4.07,15358,2024,Romance,439,Karina Halle,en,"['love', 'english']","['thirty', 'eight', 'twenty', 'three', 'speaks...",0.175129,0.475090,['romance'],romance love english thirty eight twenty three...
3,https://www.goodreads.com/book/show/31328891-s...,Stuck in the Game,https://i.gr-assets.com/images/S/compressed.ph...,After a terrible car accident puts seventeen-y...,3.84,277,59,Fantasy,226,Christopher Keene,en,"['stuck', 'game']","['terrible', 'car', 'accident', 'puts', 'seven...",-0.151667,0.438819,['fantasy'],fantasy stuck game terrible car accident puts ...
4,https://www.goodreads.com/book/show/10507293-t...,The Selection,https://i.gr-assets.com/images/S/compressed.ph...,"For thirty-five girls, the Selection is the ch...",4.12,1004868,53737,Young Adult,336,Kiera Cass,en,['selection'],"['thirty', 'five', 'girls', 'selection', 'chan...",0.126263,0.573148,['youngadult'],youngadult selection thirty five girls selecti...


In [122]:
#https://practicaldatascience.co.uk/data-science/how-to-create-content-recommendations-using-tf-idf
#bag of words is already made, but this will also down-weight common words that appear across documents
tfidf = TfidfVectorizer(stop_words='english',smooth_idf=True)

# numbers to calculate similarities
tfidf_matrix = tfidf.fit_transform(df['bag_of_words']).todense()

#calculate cosine matrix
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
print(cosine_similarities)

indices = pd.Series(df['bookTitle'])

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


[[1.         0.         0.01775768 ... 0.01067395 0.00316837 0.00516183]
 [0.         1.         0.00189812 ... 0.00523048 0.         0.02012759]
 [0.01775768 0.00189812 1.         ... 0.03741114 0.01171437 0.0072037 ]
 ...
 [0.01067395 0.00523048 0.03741114 ... 1.         0.01380771 0.0256078 ]
 [0.00316837 0.         0.01171437 ... 0.01380771 1.         0.02645521]
 [0.00516183 0.02012759 0.0072037  ... 0.0256078  0.02645521 1.        ]]


In [123]:
#get recommendations based on tf-idf and cosine_similarities

def recommend_tf(bookTitle, cosine_similarities = cosine_similarities):
    
    recommended_book = []
    recommended_book_url = []
    recommended_book_image= []
    recommended_book_author = []
    recommended_book_rating =[]
    recommended_book_descrip =[]
    recommended_book_genre =[]
    recommended_book_score = []
    
    idx = indices[indices == bookTitle].index[0]
    
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)
    
    top_10_indices = list(score_series.iloc[1:11].index)
    
    for i in top_10_indices:
        recommended_book.append(list(df['bookTitle'])[i])
        recommended_book_url.append(list(df['url'])[i])
        recommended_book_image.append(list(df['bookImage'])[i])
        recommended_book_author.append(list(df['Author'])[i])
        recommended_book_rating.append(list(df['bookRating'])[i])
        recommended_book_descrip.append(list(df['bookDesc'])[i])
        recommended_book_genre.append(list(df['Genre'])[i])
        recommended_book_score.append(score_series[i])

    data = {'Title': recommended_book,
           'Author': recommended_book_author,
            'Genre': recommended_book_genre,
            'Description' : recommended_book_descrip,
            'Rating':recommended_book_rating,
            'URL': recommended_book_url,
            'Image': recommended_book_image,
            'Score': recommended_book_score
           }
    rec_df = pd.DataFrame(data)
    
    return rec_df

In [124]:
recommend_tf('Twilight')

,Title,Author,Genre,Description,Rating,URL,Image,Score
0,Twilight Watch,Sergei Lukyanenko,Fantasy,The third book in the bestselling Russian tril...,4.12,https://www.goodreads.com/book/show/104159.Twi...,https://i.gr-assets.com/images/S/compressed.ph...,0.243253
1,The Twilight Saga: The Official Illustrated Guide,Stephenie Meyer,Young Adult,Fans of the #1 New York Times bestselling Twil...,4.25,https://www.goodreads.com/book/show/3609763-th...,https://i.gr-assets.com/images/S/compressed.ph...,0.144089
2,Breaking Dawn,Stephenie Meyer,Young Adult,"""Don't be afraid,"" I murmured. ""We belong toge...",3.70,https://www.goodreads.com/book/show/1162543.Br...,https://i.gr-assets.com/images/S/compressed.ph...,0.126752
3,A Book About Absolutely Nothing.,I.M. Nobody,Nonfiction,A book about absolutely nothing at all. A nove...,4.29,https://www.goodreads.com/book/show/17287259-a...,https://i.gr-assets.com/images/S/compressed.ph...,0.125874
4,The Twilight Saga Breaking Dawn Part 1: The Of...,Mark Cotta Vaz,Fantasy,"A companion to the film, illustrated with full...",4.28,https://www.goodreads.com/book/show/11260526-t...,https://i.gr-assets.com/images/S/compressed.ph...,0.120109
5,"Twilight: The Graphic Novel, Vol. 1",Young Kim,Sequential Art-Graphic Novels,When Isabella Swan moves to the gloomy town of...,3.86,https://www.goodreads.com/book/show/7619292-tw...,https://i.gr-assets.com/images/S/compressed.ph...,0.114458
6,Twilight Director's Notebook: The Story of How...,Catherine Hardwicke,Nonfiction,"A personal, behind-the-scenes look at the maki...",4.26,https://www.goodreads.com/book/show/6202690-tw...,https://i.gr-assets.com/images/S/compressed.ph...,0.112433
7,New Moon,Stephenie Meyer,Young Adult,There is an alternate cover edition for ISBN13...,3.55,https://www.goodreads.com/book/show/49041.New_...,https://i.gr-assets.com/images/S/compressed.ph...,0.109535
8,Eclipse,Stephenie Meyer,Young Adult,"""BELLA?""Edward's soft voice came from behind m...",3.70,https://www.goodreads.com/book/show/428263.Ecl...,https://i.gr-assets.com/images/S/compressed.ph...,0.104372
9,The Short Second Life of Bree Tanner,Stephenie Meyer,Young Adult,FANS OF THE TWILIGHT SAGA WILL BE ENTHRALLED B...,3.51,https://www.goodreads.com/book/show/7937462-th...,https://i.gr-assets.com/images/S/compressed.ph...,0.102120


## Process Model for flask app

In [125]:
import pandas as pd
import numpy as np

In [126]:
#code that stays consistent
df = pd.read_csv("../data/goodreads_final_bagowords.csv")

tfidf = TfidfVectorizer(stop_words='english',smooth_idf=True)
tfidf_matrix = tfidf.fit_transform(df['bag_of_words']).todense()
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

#make into dataframe
#df_data = pd.DataFrame(cosine_similarities).reset_index()

#string columns names required for parquet file
#df_data.columns = df_data.columns.astype(str)
#df_data.info()

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [127]:
#cosine_similarities is a numpy.ndarray
type(cosine_similarities)

numpy.ndarray

In [128]:
np.save("../data/goodreads_cos_data.npy", cosine_similarities)

In [129]:
arr1 = np.load("../data/goodreads_cos_data.npy")

In [130]:
print(arr1[0:5])

[[1.         0.         0.01775768 ... 0.01067395 0.00316837 0.00516183]
 [0.         1.         0.00189812 ... 0.00523048 0.         0.02012759]
 [0.01775768 0.00189812 1.         ... 0.03741114 0.01171437 0.0072037 ]
 [0.03369792 0.         0.0207065  ... 0.03100543 0.00838452 0.        ]
 [0.02281578 0.         0.03191378 ... 0.02690124 0.00308163 0.01410507]]


In [131]:
import s3fs

In [132]:
s3 = s3fs.S3FileSystem(anon=True)

In [133]:
arr1= np.load(s3.open("book-rec-nlp/goodreads_cos_data.npy"))

In [134]:
print(arr1[0:5])

[[1.         0.         0.01775768 ... 0.01067395 0.00316837 0.00516183]
 [0.         1.         0.00189812 ... 0.00523048 0.         0.02012759]
 [0.01775768 0.00189812 1.         ... 0.03741114 0.01171437 0.0072037 ]
 [0.03369792 0.         0.0207065  ... 0.03100543 0.00838452 0.        ]
 [0.02281578 0.         0.03191378 ... 0.02690124 0.00308163 0.01410507]]


In [135]:
#import boto3
#import io
#from botocore import UNSIGNED
#from botocore.client import Config

In [136]:
#s3client = boto3.client('s3', region_name='us-east-1', config=Config(signature_version=UNSIGNED))


In [137]:

#with io.BytesIO(s3client.get_object(Bucket="book-rec-nlp", Key="goodreads_cos_data.npy")["Body"].read()) as f:
   # f.seek(0)  # rewind the file
    #arr1 = np.load(f)

In [138]:
#print(arr1[0:5])

In [139]:


#variable
bookTitle = 'To Kill a Mockingbird'


#variable reliant code
indices = pd.Series(df['bookTitle'])

recommended_book = []
recommended_book_url = []
recommended_book_image= []
recommended_book_author = []
recommended_book_rating =[]
recommended_book_descrip =[]
recommended_book_genre =[]
recommended_book_score =[]

idx = indices[indices == bookTitle].index[0]

score_series = pd.Series(arr1[idx]).sort_values(ascending = False)

top_10_indices = list(score_series.iloc[1:11].index)

for i in top_10_indices:
    recommended_book.append(list(df['bookTitle'])[i])
    recommended_book_url.append(list(df['url'])[i])
    recommended_book_image.append(list(df['bookImage'])[i])
    recommended_book_author.append(list(df['Author'])[i])
    recommended_book_rating.append(list(df['bookRating'])[i])
    recommended_book_descrip.append(list(df['bookDesc'])[i])
    recommended_book_genre.append(list(df['Genre'])[i])
    recommended_book_score.append(score_series[i])

data = {'Title': recommended_book,
       'Author': recommended_book_author,
        'Genre': recommended_book_genre,
        'Description' : recommended_book_descrip,
        'Rating':recommended_book_rating,
        'URL': recommended_book_url,
        'Image': recommended_book_image,
        'Score': recommended_book_score
       }
rec_df = pd.DataFrame(data)
rec_df

,Title,Author,Genre,Description,Rating,URL,Image,Score
0,Go Set a Watchman,Harper Lee,Fiction,From Harper Lee comes a landmark new novel set...,3.32,https://www.goodreads.com/book/show/24817626-g...,https://i.gr-assets.com/images/S/compressed.ph...,0.200545
1,Lady of Hay,Barbara Erskine,Historical-Historical Fiction,This masterfully crafted first novel tells the...,4.06,https://www.goodreads.com/book/show/299514.Lad...,https://i.gr-assets.com/images/S/compressed.ph...,0.122968
2,Fablehaven: The Complete Series Boxed Set,Brandon Mull,Fantasy,Fablehaven fans will treasure this boxed set e...,4.60,https://www.goodreads.com/book/show/8331312-fa...,https://i.gr-assets.com/images/S/compressed.ph...,0.108974
3,Black Boy,Richard Wright,Classics,Black Boy is a classic of American autobiograp...,4.07,https://www.goodreads.com/book/show/228630.Bla...,https://i.gr-assets.com/images/S/compressed.ph...,0.100820
4,Hiroshima,John Hersey,History,"On August 6, 1945, Hiroshima was destroyed by ...",3.99,https://www.goodreads.com/book/show/27323.Hiro...,https://i.gr-assets.com/images/S/compressed.ph...,0.099956
5,The Prophet,Kahlil Gibran,Poetry,"Kahlil Gibran’s masterpiece, The Prophet, is o...",4.22,https://www.goodreads.com/book/show/2547.The_P...,https://i.gr-assets.com/images/S/compressed.ph...,0.099696
6,Our Town,Thornton Wilder,Plays,Our Town was first produced and published in 1...,3.68,https://www.goodreads.com/book/show/205476.Our...,https://i.gr-assets.com/images/S/compressed.ph...,0.098858
7,Follow Your Heart,Susanna Tamaro,Fiction,An international bestseller with tremendous wo...,3.62,https://www.goodreads.com/book/show/38913.Foll...,https://i.gr-assets.com/images/S/compressed.ph...,0.098157
8,For Colored Girls Who Have Considered Suicide ...,Ntozake Shange,Poetry,"This revolutionary, award-winning play by a la...",4.29,https://www.goodreads.com/book/show/58098.For_...,https://i.gr-assets.com/images/S/compressed.ph...,0.096893
9,Mockingbird,Kathryn Erskine,Young Adult,"In Caitlin’s world, everything is black or whi...",4.20,https://www.goodreads.com/book/show/6596547-mo...,https://i.gr-assets.com/images/S/compressed.ph...,0.092412


In [140]:
#explore variables
idx

4998

In [141]:
score_series

4998    1.000000
4483    0.200545
937     0.122968
2876    0.108974
3844    0.100820
          ...   
3561    0.000000
4328    0.000000
4329    0.000000
3563    0.000000
4346    0.000000
Length: 5000, dtype: float64

In [142]:
top_10_indices

[4483, 937, 2876, 3844, 1816, 1902, 1872, 2142, 287, 2051]

In [143]:
df_data.index

RangeIndex(start=0, stop=8863, step=1)